# VAE Training - Faces dataset

## imports

In [1]:
import os
from glob import glob
import numpy as np

from models.VAE import VariationalAutoencoder
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.
/home/comadan/.venv/gdl/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/comadan/.venv/gdl/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
# run params
section = 'vae'
run_id = '0001'
data_name = 'faces'
RUN_FOLDER = 'run/{}/'.format(section)
RUN_FOLDER += '_'.join([run_id, data_name])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' #'load' #


DATA_FOLDER = './data/celeb/'

## data

In [3]:
INPUT_DIM = (128,128,3)
BATCH_SIZE = 32

filenames = np.array(glob(os.path.join(DATA_FOLDER, '*/*.jpg')))

NUM_IMAGES = len(filenames)

In [4]:
data_gen = ImageDataGenerator(rescale=1./255)

data_flow = data_gen.flow_from_directory(DATA_FOLDER
                                         , target_size = INPUT_DIM[:2]
                                         , batch_size = BATCH_SIZE
                                         , shuffle = True
                                         , class_mode = 'input'
                                         , subset = "training"
                                            )

Found 202599 images belonging to 1 classes.


## architecture

In [5]:
vae = VariationalAutoencoder(
                input_dim = INPUT_DIM
                , encoder_conv_filters=[32,64,64, 64]
                , encoder_conv_kernel_size=[3,3,3,3]
                , encoder_conv_strides=[2,2,2,2]
                , decoder_conv_t_filters=[64,64,32,3]
                , decoder_conv_t_kernel_size=[3,3,3,3]
                , decoder_conv_t_strides=[2,2,2,2]
                , z_dim=200
                , use_batch_norm=True
                , use_dropout=True)

if mode == 'build':
    vae.save(RUN_FOLDER)
else:
    vae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [6]:
vae.encoder.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 64, 64, 32)   896         encoder_input[0][0]              
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 32)   128         encoder_conv_0[0][0]             
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 64, 64, 32)   0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [7]:
vae.decoder.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              823296    
_________________________________________________________________
reshape_1 (Reshape)          (None, 8, 8, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 16, 16, 64)        36928     
_________________________________________________________________
batch_normalization_5 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16, 16, 64)        0   

## training

In [8]:
LEARNING_RATE = 0.0005
R_LOSS_FACTOR = 10000
EPOCHS = 200
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

In [9]:
vae.compile(LEARNING_RATE, R_LOSS_FACTOR)

In [10]:
vae.train_with_generator(     
    data_flow
    , epochs = EPOCHS
    , steps_per_epoch = NUM_IMAGES / BATCH_SIZE
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)



Epoch 1/200
6332/6331 [==============================] - 263s 41ms/step - loss: 275.9504 - vae_r_loss: 214.1513 - vae_kl_loss: 61.7991

Epoch 00001: saving model to run/vae/0001_faces/weights/weights-001-275.95.h5

Epoch 00001: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 2/200
6332/6331 [==============================] - 266s 42ms/step - loss: 233.4913 - vae_r_loss: 174.8540 - vae_kl_loss: 58.6373

Epoch 00002: saving model to run/vae/0001_faces/weights/weights-002-233.48.h5

Epoch 00002: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 3/200
6332/6331 [==============================] - 263s 41ms/step - loss: 228.4519 - vae_r_loss: 169.3951 - vae_kl_loss: 59.0568

Epoch 00003: saving model to run/vae/0001_faces/weights/weights-003-228.45.h5

Epoch 00003: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 4/200
6332/6331 [==============================] - 259s 41ms/step - loss: 226.2200 - vae_r_loss: 167.0799 - vae_kl_loss: 59.1402

Epoch 00004: sav


Epoch 00028: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 29/200
6332/6331 [==============================] - 260s 41ms/step - loss: 218.9233 - vae_r_loss: 159.9414 - vae_kl_loss: 58.9819

Epoch 00029: saving model to run/vae/0001_faces/weights/weights-029-218.92.h5

Epoch 00029: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 30/200
6332/6331 [==============================] - 259s 41ms/step - loss: 218.8397 - vae_r_loss: 159.8679 - vae_kl_loss: 58.9719

Epoch 00030: saving model to run/vae/0001_faces/weights/weights-030-218.84.h5

Epoch 00030: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 31/200
6332/6331 [==============================] - 258s 41ms/step - loss: 218.8273 - vae_r_loss: 159.8532 - vae_kl_loss: 58.9741

Epoch 00031: saving model to run/vae/0001_faces/weights/weights-031-218.83.h5

Epoch 00031: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 32/200
6332/6331 [==============================] - 260s 41ms/step - loss: 21


Epoch 00057: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 58/200
6332/6331 [==============================] - 260s 41ms/step - loss: 217.9086 - vae_r_loss: 158.9259 - vae_kl_loss: 58.9826

Epoch 00058: saving model to run/vae/0001_faces/weights/weights-058-217.91.h5

Epoch 00058: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 59/200
6332/6331 [==============================] - 259s 41ms/step - loss: 217.8565 - vae_r_loss: 158.8833 - vae_kl_loss: 58.9731

Epoch 00059: saving model to run/vae/0001_faces/weights/weights-059-217.86.h5

Epoch 00059: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 60/200
6332/6331 [==============================] - 259s 41ms/step - loss: 217.8350 - vae_r_loss: 158.8685 - vae_kl_loss: 58.9665

Epoch 00060: saving model to run/vae/0001_faces/weights/weights-060-217.83.h5

Epoch 00060: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 61/200
6332/6331 [==============================] - 258s 41ms/step - loss: 21

Epoch 87/200
6332/6331 [==============================] - 259s 41ms/step - loss: 217.4660 - vae_r_loss: 158.5033 - vae_kl_loss: 58.9627

Epoch 00087: saving model to run/vae/0001_faces/weights/weights-087-217.47.h5

Epoch 00087: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 88/200
6332/6331 [==============================] - 259s 41ms/step - loss: 217.4683 - vae_r_loss: 158.5062 - vae_kl_loss: 58.9621

Epoch 00088: saving model to run/vae/0001_faces/weights/weights-088-217.46.h5

Epoch 00088: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 89/200
6332/6331 [==============================] - 260s 41ms/step - loss: 217.4924 - vae_r_loss: 158.5159 - vae_kl_loss: 58.9765

Epoch 00089: saving model to run/vae/0001_faces/weights/weights-089-217.48.h5

Epoch 00089: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 90/200
6332/6331 [==============================] - 259s 41ms/step - loss: 217.4810 - vae_r_loss: 158.4833 - vae_kl_loss: 58.9977

Epoch 00090: s

6332/6331 [==============================] - 258s 41ms/step - loss: 217.2189 - vae_r_loss: 158.2551 - vae_kl_loss: 58.9638

Epoch 00116: saving model to run/vae/0001_faces/weights/weights-116-217.22.h5

Epoch 00116: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 117/200
6332/6331 [==============================] - 259s 41ms/step - loss: 217.2268 - vae_r_loss: 158.2516 - vae_kl_loss: 58.9752

Epoch 00117: saving model to run/vae/0001_faces/weights/weights-117-217.23.h5

Epoch 00117: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 118/200
6332/6331 [==============================] - 259s 41ms/step - loss: 217.1783 - vae_r_loss: 158.2328 - vae_kl_loss: 58.9455

Epoch 00118: saving model to run/vae/0001_faces/weights/weights-118-217.18.h5

Epoch 00118: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 119/200
6332/6331 [==============================] - 260s 41ms/step - loss: 217.2394 - vae_r_loss: 158.2402 - vae_kl_loss: 58.9992

Epoch 00119: saving mode

6332/6331 [==============================] - 259s 41ms/step - loss: 217.0910 - vae_r_loss: 158.0997 - vae_kl_loss: 58.9914

Epoch 00145: saving model to run/vae/0001_faces/weights/weights-145-217.09.h5

Epoch 00145: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 146/200
6332/6331 [==============================] - 260s 41ms/step - loss: 217.0725 - vae_r_loss: 158.1030 - vae_kl_loss: 58.9695

Epoch 00146: saving model to run/vae/0001_faces/weights/weights-146-217.07.h5

Epoch 00146: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 147/200
6332/6331 [==============================] - 261s 41ms/step - loss: 217.1142 - vae_r_loss: 158.1120 - vae_kl_loss: 59.00221s - loss: 217.1145 - vae_r_loss: 158.10

Epoch 00147: saving model to run/vae/0001_faces/weights/weights-147-217.12.h5

Epoch 00147: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 148/200
6332/6331 [==============================] - 260s 41ms/step - loss: 217.0953 - vae_r_loss: 158.0944 - vae_kl

Epoch 174/200
6332/6331 [==============================] - 260s 41ms/step - loss: 217.0154 - vae_r_loss: 158.0063 - vae_kl_loss: 59.0091

Epoch 00174: saving model to run/vae/0001_faces/weights/weights-174-217.02.h5

Epoch 00174: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 175/200
6332/6331 [==============================] - 260s 41ms/step - loss: 216.9653 - vae_r_loss: 157.9387 - vae_kl_loss: 59.0266

Epoch 00175: saving model to run/vae/0001_faces/weights/weights-175-216.96.h5

Epoch 00175: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 176/200
6332/6331 [==============================] - 260s 41ms/step - loss: 216.9254 - vae_r_loss: 157.9235 - vae_kl_loss: 59.0019

Epoch 00176: saving model to run/vae/0001_faces/weights/weights-176-216.93.h5

Epoch 00176: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 177/200
6332/6331 [==============================] - 261s 41ms/step - loss: 217.0008 - vae_r_loss: 157.9662 - vae_kl_loss: 59.0345

Epoch 0017